# Setup

In [1]:
import numpy as np
import librosa
import torch 
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from datetime import datetime
import pdb 
from google.colab import drive
import pandas as pd
from psutil import virtual_memory
import os
from zipfile import ZipFile
import torchvision
from collections import defaultdict
from torchvision import transforms
!pip install wandb --upgrade
import wandb
from sklearn import metrics
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
wandb.login()

wandb: Currently logged in as: nicholasmagal. Use `wandb login --relogin` to force relogin


True

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(0)

#Model

In [4]:
#hyperparameters
config = {"learning rate": .001,
          "batch size": 256,
          "epochs": 15,
          'patience': 4}

In [5]:
#Gpu support
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device is on",device)

#Getting Resnet and loading pretrained weights depending on mode
def model_creator(pretrained, pretrained_model_path = None):
  if pretrained == False:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
    model.fc = nn.Linear(512, 257)
    print("random init")

  elif pretrained == True:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
    model.fc = nn.Linear(512, 50)
    model.load_state_dict(torch.load(pretrained_model_path, map_location=torch.device('cpu')))
    model.fc = nn.Linear(512, 257) 

    #Freezing first 5 layers
    for i, child in enumerate(model.children()):
      if i == 5:
        break
      for param in child.parameters():
        param.requires_grad = False

    print('loaded pretrained')

  #creating optimizer 
  optimizer = torch.optim.Adam(model.parameters(), lr=config['learning rate'])
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',
                                                       patience = config['patience'])
  return model, optimizer, scheduler

#loss 
loss_function = torch.nn.CrossEntropyLoss()


Device is on cuda:0


#Data

In [6]:
import os
import os.path
from typing import Any, Callable, List, Optional, Tuple, Union

from PIL import Image

from torchvision.datasets.utils import download_and_extract_archive, verify_str_arg
from torchvision.datasets import VisionDataset

class Caltech256(VisionDataset):
    """`Caltech 256 <https://data.caltech.edu/records/20087>`_ Dataset.

    Args:
        root (string): Root directory of dataset where directory
            ``caltech256`` exists or will be saved to if download is set to True.
        transform (callable, optional): A function/transform that takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """

    def __init__(
        self,
        root: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        download: bool = False,
    ) -> None:
        super().__init__(os.path.join(root, "caltech256"), transform=transform, target_transform=target_transform)
        os.makedirs(self.root, exist_ok=True)

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError("Dataset not found or corrupted. You can use download=True to download it")

        self.categories = sorted(os.listdir(os.path.join(self.root, "256_ObjectCategories")))
        self.index: List[int] = []
        self.y = []
        for (i, c) in enumerate(self.categories):
            n = len(
                [
                    item
                    for item in os.listdir(os.path.join(self.root, "256_ObjectCategories", c))
                    if item.endswith(".jpg")
                ]
            )
            self.index.extend(range(1, n + 1))
            self.y.extend(n * [i])

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img = Image.open(
            os.path.join(
                self.root,
                "256_ObjectCategories",
                self.categories[self.y[index]],
                f"{self.y[index] + 1:03d}_{self.index[index]:04d}.jpg",
            )
        ).convert("RGB")

        target = self.y[index]

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target
        
    def _check_integrity(self) -> bool:
        # can be more robust and check hash of files
        return os.path.exists(os.path.join(self.root, "256_ObjectCategories"))

    def __len__(self) -> int:
        return len(self.index)

    def download(self) -> None:
        if self._check_integrity():
            print("Files already downloaded and verified")
            return

        download_and_extract_archive(
            "https://drive.google.com/file/d/1r6o0pSROcV1_VwT4oSjA2FBUSCWGuxLK",
            self.root,
            filename="256_ObjectCategories.tar",
            md5="67b4f42ca05d46448c6bb8ecd2220f6d",
        )



In [8]:
#Dataloader and Datasets
cal_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = Caltech256(root = '/content', download = True, transform = cal_transform)
generator1 = torch.Generator().manual_seed(42)

training_dataset, validation_dataset, testing_dataset = torch.utils.data.random_split(dataset, [.6,.2,.2], generator = generator1)

training_loader = torch.utils.data.DataLoader(training_dataset,
                                                   batch_size = config['batch size'],
                                                   shuffle = True,
                                                   num_workers = 1,
                                                   pin_memory = True)

validation_loader = torch.utils.data.DataLoader(validation_dataset,
                                                   batch_size = config['batch size'],
                                                   shuffle = True,
                                                   num_workers = 1,
                                                   pin_memory = True)

test_loader = torch.utils.data.DataLoader(testing_dataset,
                                                   batch_size = config['batch size'],
                                                   shuffle = False,
                                                   num_workers = 1,
                                                   pin_memory = True)


Files already downloaded and verified


#Training/Testing

In [9]:
#train/validate network
def train_validate_test(model, train_loader, val_loader, optimizer, loss_function, total_epochs, device, scheduler, config, model_save_location, test_loader, name):

  wandb.init(project="resnet_pretrained_caltech_run_frozen_weights", name = name, config=config)

  best_validation_acc = -float('inf')
  for epoch_num in range(total_epochs):
    starting_time = datetime.now()
    #summary stats
    train_loss_tracker = 0
    validation_loss_tracker = 0
    total_correct_train = 0
    total_correct_validation = 0

    model.train() 
    
    for x, y in train_loader:
      optimizer.zero_grad()
    
      x, y = x.to(device), y.to(device) 
      train_output = model(x)

      train_loss = loss_function(train_output, y.long())
      train_loss.backward()
      optimizer.step()
      
      #storing loss
      train_loss_tracker += train_loss.item()

      #storing for acc
      prediction_logits, prediction_classes = torch.max(train_output,1)
      total_correct_train += (prediction_classes==y).sum().item()
    
    with torch.no_grad():
      model.eval()
      
      #storage for evaluation of performance
      ground_truth = []
      predictions = []
      
      for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        
        #Calculating Loss
        val_output = model(x)
        val_loss = loss_function(val_output, y.long())
        validation_loss_tracker+= val_loss.item()

        #Calculating Accuracy
        prediction_logits, prediction_classes = torch.max(val_output,1)
        total_correct_validation += (prediction_classes==y).sum().item()

        #Storage for AUC ROC
        val_output = torch.nn.functional.softmax(val_output, dim = 1)
        ground_truth.append(y.cpu().detach().numpy())
        predictions.append(val_output.cpu().detach().numpy())

    
    #computing summary stats 
    avg_train_loss = train_loss_tracker/len(train_loader.dataset)
    avg_train_acc = total_correct_train/len(train_loader.dataset)
    avg_val_loss = (validation_loss_tracker/len(val_loader.dataset))
    avg_val_acc = (total_correct_validation/len(val_loader.dataset))

    #roc auc
    ground_truth = np.concatenate((ground_truth), axis = 0)
    predictions = np.concatenate((predictions), axis = 0)
    roc_auc = metrics.roc_auc_score(ground_truth, predictions, average='macro', multi_class = 'ovo')
    
    scheduler.step(avg_val_loss)
    
    epoch_runtime = datetime.now() - starting_time 
    print(f'Epoch: {epoch_num} Train Loss: {avg_train_loss:.4f} Val Loss: {avg_val_loss:.4f} Train Acc: {avg_train_acc:.4f} Val Accuracy: {avg_val_acc:.4f} Val ROC AUC {roc_auc:.4f}')
    print(f'Epoch Runtime {epoch_runtime}')
    wandb.log({"Training Average Loss":avg_train_loss, 'Training Avg Acc':avg_train_acc, "Val Avg Loss":avg_val_loss, "Val Avg Acc":avg_val_acc, "Val ROC AUC":roc_auc})

    #if our loss is the new best metric save the model to load later
    if avg_val_acc > best_validation_acc:
      best_validation_acc = avg_val_acc
      torch.save(model.state_dict(), model_save_location)
      print("saving model")
  
  def test(test_loader, model, device):
    with torch.no_grad():
      model.eval()

      #storage for evaluation of performance
      ground_truth = []
      predictions = []
      total_correct_test = 0

      for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        
        #Feedforward
        test_output = model(x)

        #Accuracy Storage
        prediction_logits, prediction_classes = torch.max(test_output,1)
        total_correct_test += (prediction_classes==y).sum().item()

        #updating for ROC function 
        test_output = torch.nn.functional.softmax(test_output, dim = 1)
        ground_truth.append(y.cpu().detach().numpy())
        predictions.append(test_output.cpu().detach().numpy())

      ground_truth = np.concatenate((ground_truth), axis = 0)
      predictions = np.concatenate((predictions), axis = 0)
      roc_auc = metrics.roc_auc_score(ground_truth, predictions, average='macro', multi_class = 'ovo')
      acc = total_correct_test/len(test_loader.dataset)

      wandb.log({"Test Acc":acc, "Test ROC AUC":roc_auc })
      return roc_auc, acc
  #Load best performing model
  model.load_state_dict(torch.load(model_save_location))
  test_roc, test_acc = test(test_loader, model, device)
  wandb.finish()

In [10]:
#Train on variable lengths of data, also swtiching between pretrained and random init models

#percent of data to train on
training_percentage = [.05, .15, .30, .50, .70, 1]
for percentage in training_percentage:

  config['data amount'] = percentage

  #adjusting training data based off of percentage 
  total_training_samples = int(percentage * len(training_dataset))
  print("training with: ", total_training_samples)
  new_training_dataset = torch.utils.data.Subset(training_dataset, list(range(total_training_samples)))

  training_loader = torch.utils.data.DataLoader(new_training_dataset,
                                                    batch_size = config['batch size'],
                                                    shuffle = True,
                                                    num_workers = 1,
                                                    pin_memory = True)

  #for each percentage we must train both the pretrained model as well as the randomly init model
  for i in range(2):
    
    #load random init model for i == 0 
    if i == 0:
      config['Pretrained'] = False
      model, optimizer, scheduler = model_creator(config['Pretrained'])
      
    #Otherwise load pretrained model
    elif i == 1:
      pretrained_save_path = '/content/drive/MyDrive/transfer_learning_audio_to_image/saves/.87 .001 lr/resent_audio_pretrained'
      config['Pretrained'] = True
      model, optimizer, scheduler = model_creator(config['Pretrained'], pretrained_save_path)
      
    model.to(device)
    model_save_location = '/content/drive/MyDrive/transfer_learning_audio_to_image/saves/resent50_caltech'+ str(percentage) + str(config['Pretrained'])

    #Training and Testing
    run_name = 'pretrained_'+ str(percentage) if config['Pretrained'] == True else 'random_init_'+ str(percentage)
    train_validate_test(model, training_loader, validation_loader, optimizer, loss_function, config['epochs'], device,scheduler, config,model_save_location, test_loader, run_name)

training with:  918
random init


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch: 0 Train Loss: 0.0245 Val Loss: 0.0647 Train Acc: 0.0251 Val Accuracy: 0.0033 Val ROC AUC 0.5040
Epoch Runtime 0:01:32.270293
saving model
Epoch: 1 Train Loss: 0.0218 Val Loss: 0.1085 Train Acc: 0.0566 Val Accuracy: 0.0036 Val ROC AUC 0.5069
Epoch Runtime 0:01:28.402468
saving model
Epoch: 2 Train Loss: 0.0200 Val Loss: 0.0690 Train Acc: 0.0937 Val Accuracy: 0.0060 Val ROC AUC 0.5211
Epoch Runtime 0:01:27.442258
saving model
Epoch: 3 Train Loss: 0.0187 Val Loss: 0.0625 Train Acc: 0.1133 Val Accuracy: 0.0109 Val ROC AUC 0.5227
Epoch Runtime 0:01:27.488540
saving model
Epoch: 4 Train Loss: 0.0174 Val Loss: 0.0344 Train Acc: 0.1242 Val Accuracy: 0.0206 Val ROC AUC 0.5754
Epoch Runtime 0:01:27.285986
saving model
Epoch: 5 Train Loss: 0.0162 Val Loss: 0.0342 Train Acc: 0.1863 Val Accuracy: 0.0209 Val ROC AUC 0.5859
Epoch Runtime 0:01:28.470978
saving model
Epoch: 6 Train Loss: 0.0151 Val Loss: 0.0281 Train Acc: 0.2070 Val Accuracy: 0.0356 Val ROC AUC 0.6273
Epoch Runtime 0:01:27.13773

Test Acc,▁
Test ROC AUC,▁
Training Average Loss,█▆▅▅▄▃▂▂▂▁▁▁▁▁▁
Training Avg Acc,▁▂▃▃▃▅▅▆▇▇▇████
Val Avg Acc,▁▁▁▂▂▂▃▅▆▇█████
Val Avg Loss,▄█▅▄▂▂▁▁▁▁▁▁▁▁▁
Val ROC AUC,▁▁▂▂▄▄▆▇▇██████
Test Acc,0.09982
Test ROC AUC,0.68265
Training Average Loss,0.01256
Training Avg Acc,0.33224


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


...

Test Acc,▁
Test ROC AUC,▁
Training Average Loss,█▆▅▅▄▄▂▂▂▁▁▁▁▁▁
Training Avg Acc,▁▂▃▃▄▄▅▆▇▇▇████
Val Avg Acc,▁▃▄▅▅▅▇▇▇▇█████
Val Avg Loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Val ROC AUC,▁▅▆▆▇▇█████████
Test Acc,0.19033
Test ROC AUC,0.83623
Training Average Loss,0.01322
Training Avg Acc,0.27954


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


loaded pretrained


Epoch: 0 Train Loss: 0.0213 Val Loss: 0.0301 Train Acc: 0.0528 Val Accuracy: 0.0286 Val ROC AUC 0.5383
Epoch Runtime 0:01:56.887341
saving model
Epoch: 1 Train Loss: 0.0193 Val Loss: 0.0279 Train Acc: 0.0850 Val Accuracy: 0.0271 Val ROC AUC 0.5393
Epoch Runtime 0:01:57.775576
Epoch: 2 Train Loss: 0.0181 Val Loss: 0.0216 Train Acc: 0.1096 Val Accuracy: 0.0569 Val ROC AUC 0.6852
Epoch Runtime 0:01:56.760011
saving model
Epoch: 3 Train Loss: 0.0171 Val Loss: 0.0181 Train Acc: 0.1360 Val Accuracy: 0.1225 Val ROC AUC 0.7950
Epoch Runtime 0:01:56.231376
saving model
Epoch: 4 Train Loss: 0.0163 Val Loss: 0.0177 Train Acc: 0.1597 Val Accuracy: 0.1382 Val ROC AUC 0.8090
Epoch Runtime 0:01:56.994121
saving model
Epoch: 5 Train Loss: 0.0156 Val Loss: 0.0173 Train Acc: 0.1795 Val Accuracy: 0.1521 Val ROC AUC 0.8198
Epoch Runtime 0:01:56.127198
saving model
Epoch: 6 Train Loss: 0.0145 Val Loss: 0.0166 Train Acc: 0.2213 Val Accuracy: 0.1732 Val ROC AUC 0.8327
Epoch Runtime 0:01:56.521708
saving mode

KeyboardInterrupt: ignored